# Прогнозирование на основе обученной модели классификации клиентов ЭДО 
# (Prediction based on a trained Diadok client classification model)

### 1. Подготовка и обработка датасета (Preparation and preprocessing of dataset)

In [1]:
# standart library block 
import pandas as pd
import numpy as np
import datetime

In [2]:
# Чтение файла с данными (Reading data file)
dd_data = pd.read_excel("DD_емкость.xlsx", sheet_name='Основная OBS', dtype = {'inn': object, 'kpp': object, 'okved': object,\
                                                                               'CLC': object, 'cnt_Kontur_product': object})

In [3]:
# Информация о датасете (Info about dataset)
dd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13732 entries, 0 to 13731
Data columns (total 63 columns):
inn                               13732 non-null object
kpp                               13732 non-null object
otrasl                            13732 non-null float64
segment                           13732 non-null float64
okved                             13732 non-null object
InvitesOut                        13732 non-null int64
InvitesOutAccepted                13732 non-null int64
InvitesIn                         13732 non-null int64
InvitesInAccepted                 13732 non-null int64
InvitesInALLTime                  13732 non-null int64
InvitesNoInYear                   13732 non-null int64
SearchesOut                       13732 non-null int64
SearchesIn                        13732 non-null int64
s1                                13732 non-null int64
trafficTop                        13732 non-null int64
v5portrait                        13732 non-null int64
top_

In [4]:
# Преобразование данных в числовой формат. Замена значений NaN на 0.
pd.set_option('display.max_rows', 63)
#dd_data = dd_data.apply(pd.to_numeric)
#dd_data.fillna(0) #преобразование значений Nan в 0
dd_data.tail().T

,13727,13728,13729,13730,13731
inn,2312177021,2312227850,2311159245,2315074793,2635045294
kpp,231201001,231201001,231101001,231501001,237201001
otrasl,0.02039,0.04076,0.04076,0.00725,0.00792
segment,0.01805,0.02354,0.02354,0.00107,0.00299
okved,33.1,46.7,46.7,46.4,46.7
InvitesOut,1,0,0,0,4
InvitesOutAccepted,1,0,0,0,3
InvitesIn,2,2,1,1,3
InvitesInAccepted,40,1,0,4,2
InvitesInALLTime,40,1,0,4,2


In [5]:
# Размер датасета (Dataset size)
dd_data.shape

(13732, 63)

### 2. Прогнозирование на основе обученной модели (Prediction based on a trained model)

In [6]:
# standart library block
import numpy as np
import pandas as pd
import pickle

# learning library block
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

from catboost import CatBoostClassifier
from catboost import cv
from catboost.datasets import amazon
from catboost.datasets import epsilon

#### Подготовка датасета (Prepared dataset)

In [7]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки. 
# (Purpose of "target", "feature". Distribution of data on training and test samples.)
z=dd_data['inn'].copy()
X=dd_data.drop(['DD_service', 'inn','kpp','DD_bill_date', 'DD_bill_sum',\
                'DD_pay_date', 'DD_pay_sum', 'DD_since', 'DD_upto', 'CLC',  'DD_service'], axis=1)

In [8]:
# Стандартизация значений переменных (Standardization of features values)
scaler = StandardScaler()
scaler.fit_transform(X)
X_scaled = scaler.transform(X)

C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


#### Прогнозирование - Модель RandomForest ver.1 (Prediction - RandomForest ver.1)

In [9]:
model_clf = RandomForestClassifier() # выбираем тип модели (choose the type of model)
model_clf = joblib.load('dd_clf_ver.1.pkl') # загружаем обученную модель (load trained model)

dd_pred_clf_train = model_clf.predict_proba(X)[:,1] # предсказание для рабочего датасета (prediction for work sample)

#### Прогнозирование - Модель GradientBoostingClassifier  ver.1 (Prediction - GradientBoostingClassifier  ver.1)

In [10]:
model_gbclf = GradientBoostingClassifier() # выбираем тип модели (choose the type of model)
model_gbclf = joblib.load('dd_gbclf_ver.1.pkl') # загружаем обученную модель (load trained model)

dd_pred_gbclf_train = model_gbclf.predict_proba(X)[:,1] # предсказание для рабочего датасета (prediction for work sample)

#### Прогнозирование - Модель CatBoost ver.1 (Prediction - Model CatBoost ver.1)

In [11]:
model = CatBoostClassifier() # выбираем тип модели (choose the type of model)
model.load_model('dd_cat_ver.1.cbm') # загружаем обученную модель (load trained model)

dd_pred_cat_train = model.predict_proba(data=X)[:,1] # предсказание для рабочего датасета (prediction for work sample)

#### Сохранение результатов

In [12]:
df1 = pd.DataFrame(z)

In [13]:
# Сохранение полученных резульатов модель RandomForest (RandomForest save the results)
df2_1 = pd.DataFrame(dd_pred_clf_train)
df2_1.columns=['Predictions']
res_clf = pd.concat([df1, df2_1.reindex(df1.index)], axis=1)
predict_results_clf = pd.DataFrame(res_clf).to_csv('prediction_dd_clf_model.csv', index=False, sep=';')

In [14]:
# Сохранение полученных резульатов модель GradientBoosting (GradientBoosting save the results)
df2_2 = pd.DataFrame(dd_pred_gbclf_train)
df2_2.columns=['Predictions']
res_gbclf = pd.concat([df1, df2_2.reindex(df1.index)], axis=1)
predict_results_gbclf = pd.DataFrame(res_gbclf).to_csv('prediction_dd_gbclf_model.csv', index=False, sep=';')

In [15]:
# Сохранение полученных резульатов модель CatBoost (CatBoost save the results)
df2_3 = pd.DataFrame(dd_pred_cat_train)
df2_3.columns=['Predictions']
#res_cat = pd.concat([df1, df2_3.reindex(df1.index)], axis=1)
predict_results_cat = pd.DataFrame(df2_3).to_csv('prediction_dd_cat_model.csv', index=True, sep=';')